# Example of using the Borehole3D class

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.omf import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon import Lexicon_FR_updated as lexicon_fr
from utils.io import boreholes_from_files, striplog_from_text,gdf_viewer,\
df_from_sources, striplog_from_df
from core.core import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
from os import remove
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [2]:
save_dir = ROOT_DIR+'/CF_data/Donnees_fusionnees/'

In [3]:
df = pd.read_csv(save_dir+'Pilote_test.csv', sep=',')

In [4]:
gdf_viewer(df)

Rows : 87, columns : 9


interactive(children=(IntSlider(value=10, description='rows', max=87, min=10, readout=False), IntSlider(value=…

## Data Checking

In [5]:
df.drop(index=df.query('X!=X').index, inplace=True)

In [6]:
df = df[:-1].copy()

In [7]:
df.loc[1, 'Description'] ='Sable rouge'

In [8]:
df.query('ID=="F201"')

,ID,X,Y,Z,Litho_top,Description,Litho_base,Diam_for,Long_for
0,F201,152890.245758,122596.47426,101.926886,0.6,Remblais avec briques de construction couleur ...,1.2,75.0,2.4
1,F201,152890.245758,122596.47426,101.926886,1.2,Sable rouge,2.4,75.0,2.4


In [9]:
df.drop(index=df.query('ID=="F201"').index, inplace=True)

## Create Boreholes from dataframes or files

In [10]:
bhs_df=[df] # list of dataframes

In [11]:
boreholes, components, link_intv_comp = boreholes_from_files(bhs_df, verbose=True,
                                                litho_field='Description',
                                                lexicon='fr', #Lexicon.default(),
                                                diam_field='Diam_for', 
                                                litho_top_field='Litho_top',
                                                litho_base_field='Litho_base',
                                                thick_field='Long_for')


Dataframe 0 processing...
|__ID:'F205'
Error : No lithology matching with 'Couche de boue en surface liée à l'intervention de Géosonda' in given lexicon
Summary : [Striplog(7 Intervals, start=0.0, stop=4.8)]
{0: {'description': 'white sand', 'interval_number': 0, 'top': <core.orm.PositionOrm object at 0x7f5a38011250>, 'base': <core.orm.PositionOrm object at 0x7f5a38011550>}, 1: {'description': 'Limon brun ', 'interval_number': 1, 'top': <core.orm.PositionOrm object at 0x7f5a38027d30>, 'base': <core.orm.PositionOrm object at 0x7f5a38027c10>}, 2: {'description': 'Remblais gris gravillonneux avec présence de déchets de construction', 'interval_number': 2, 'top': <core.orm.PositionOrm object at 0x7f5a38027550>, 'base': <core.orm.PositionOrm object at 0x7f5a38027970>}, 3: {'description': "Présence d'eau avec remblais pollués (odeur forte de naphtalène avec irrisation de l'eau) + mélange de limon et gravillons ", 'interval_number': 3, 'top': <core.orm.PositionOrm object at 0x7f5a38027c40>, 

/home/yanathan/Projects/GSDMA/utils/io.py:518: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   The 'overlaps' parameter may be used to remove this warning.
  boreholes.append(BoreholeOrm(id=bh_name))
/home/yanathan/Projects/GSDMA/utils/io.py:518: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.int_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Intervals.id to

KeyError: 0

In [ ]:
df.query('ID=="F205"')

In [ ]:
boreholes

In [ ]:
components

In [ ]:
len(boreholes), len(components)

## Create a project and save data in a database

In [ ]:
remove('tmp_files/tfe_orm_db.db')

In [ ]:
engine = create_engine('sqlite:///tmp_files/tfe_orm_db.db', echo=True)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
p = Project(session)
p.add_components(components)

Add boreholes into the project

In [ ]:
for bh in boreholes:
    p.add_borehole(bh)

In [ ]:
p.add_link_between_components_and_intervals(link_intv_comp)

In [ ]:
p.commit()
print('Boreholes in the project : ', len(p.boreholes))

In [ ]:
p.boreholes

In [ ]:
p.refresh(verbose=True)

In [ ]:
session.close()

# 3D Display

## Display one object

In [ ]:
bh = p.boreholes_3d
#print(bh.name, '\n', bh.intervals) # problem with top and base position --> kind of overlaying

In [ ]:
p.boreholes_3d[0].plot2d()

In [ ]:
p.boreholes_3d[0].plot3d()

In [ ]:
p.boreholes_3d[0].plot3d(x3d=True)

### Display project boreholes

In [ ]:
p.plot3d(labels_size=15, bg_color=["royalblue", "aliceblue"], window_size=(1300, 1000))

### Boreholes in 3D using X3D for interaction within the notebook